In [1]:
import os
import yaml
import shutil
import subprocess

In [ ]:
DATA_ZIP_PATH = 'finetuned.zip.zip'
CUSTOM_DATA_DIR = 'custom_data'
DATA_YAML_PATH = 'data.yaml'

In [3]:
TRAIN_VALIDATION_RATIO = 0.9 # 90% for training, 10% for validation


In [ ]:
# Training Parameters
YOLO_MODEL = 'yolov11n.pt' 
EPOCHS = 60
IMAGE_SIZE = 640

In [ ]:
print(">>> Step 2: Unzipping the dataset...")
if not os.path.exists(CUSTOM_DATA_DIR):
    # The command is adapted for cross-platform use.
    # On Windows, you might need to install 'unzip' or use a Python library.
    # For simplicity, this uses the shell.
    try:
        subprocess.run(f"unzip -q {DATA_ZIP_PATH} -d {CUSTOM_DATA_DIR}", shell=True, check=True)
        print(f"Successfully unzipped {DATA_ZIP_PATH} to {CUSTOM_DATA_DIR}")
    except subprocess.CalledProcessError as e:
        print(f"Error unzipping file: {e}. Make sure 'unzip' is installed and in your PATH.")
        exit()
else:
    print(f"Directory '{CUSTOM_DATA_DIR}' already exists. Skipping unzip.")


In [ ]:
print("\n>>> Step 3: Splitting data into training and validation sets...")
# This step assumes 'train_val_split.py' is in the same directory.
try:
    subprocess.run(
        f"python train_val_split.py --datapath=\"{CUSTOM_DATA_DIR}\" --train_pct={TRAIN_VALIDATION_RATIO}",
        shell=True,
        check=True
    )
    print("Data successfully split.")
except subprocess.CalledProcessError as e:
    print(f"Error running train_val_split.py: {e}")
    exit()

In [ ]:
print("\n>>> Step 4: Creating the data.yaml configuration file...")
path_to_classes_txt = os.path.join(CUSTOM_DATA_DIR, 'classes.txt')

if not os.path.exists(path_to_classes_txt):
    print(f"Error: 'classes.txt' not found in '{CUSTOM_DATA_DIR}'!")
    exit()

with open(path_to_classes_txt, 'r') as f:
    classes = [line.strip() for line in f if line.strip()]
    number_of_classes = len(classes)
    print(f"Found classes: {classes}")

In [ ]:
data_config = {
    'path': os.path.abspath('data'), # Absolute path is more robust for YOLO
    'train': 'train/images',
    'val': 'validation/images',
    'nc': number_of_classes,
    'names': classes
}

with open(DATA_YAML_PATH, 'w') as f:
    yaml.dump(data_config, f, sort_keys=False)

print(f"'{DATA_YAML_PATH}' created successfully.")
print("\nFile contents:\n")
with open(DATA_YAML_PATH, 'r') as f:
    print(f.read())

In [ ]:
print("\n>>> Step 5: Starting model training...")
# The 'yolo' command should be available if 'ultralytics' is installed.
try:
    command = (
        f"yolo detect train data={DATA_YAML_PATH} "
        f"model={YOLO_MODEL} epochs={EPOCHS} imgsz={IMAGE_SIZE}"
    )
    subprocess.run(command, shell=True, check=True)
    print("Training complete.")
except subprocess.CalledProcessError as e:
    print(f"Error during training: {e}")
    exit()

In [ ]:
print("\n>>> Step 6: Testing the trained model on validation images...")
path_to_best_weights = 'runs/detect/train/weights/best.pt'
validation_images_path = os.path.join('data', 'validation', 'images')

if os.path.exists(path_to_best_weights):
    try:
        command = (
            f"yolo detect predict model={path_to_best_weights} "
            f"source={validation_images_path} save=True"
        )
        subprocess.run(command, shell=True, check=True)
        print("\nPrediction complete. Results are saved in 'runs/detect/predict/'.")
    except subprocess.CalledProcessError as e:
        print(f"Error during prediction: {e}")
else:
    print(f"Could not find trained weights at '{path_to_best_weights}'. Skipping test.")

In [ ]:
print("\n>>> Step 7: Packaging the model for deployment...")
output_model_dir = 'my_model'
if os.path.exists(output_model_dir):
    shutil.rmtree(output_model_dir) # Clean up old directory
os.makedirs(output_model_dir)

In [ ]:
shutil.copy(path_to_best_weights, os.path.join(output_model_dir, 'my_model.pt'))

In [ ]:
shutil.copytree('runs/detect/train', os.path.join(output_model_dir, 'training_results'))